In [1]:
import pandas as pd
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import time
from IPython.display import clear_output
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

In [21]:
#Preprocess training set
data=pd.read_csv('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set/labels.csv')

labelsTrain = data["\teye_color\tface_shape\tfile_name"].str.split(pat="\t", n=-1, expand=True)
labelsTrain.drop(columns =[0,1], inplace = True)
labelsTrain.columns = [ "face_shape" , "img_name"]
labelsTrain = labelsTrain.astype({'face_shape': 'int32'})
labelsTrain = labelsTrain.astype({'img_name': 'string'})

#Preprocess testing set
data=pd.read_csv('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set_test/labels.csv')

labelsTest = data["\teye_color\tface_shape\tfile_name"].str.split(pat="\t", n=-1, expand=True)
labelsTest.drop(columns =[0,1], inplace = True)
labelsTest.columns = [ "face_shape" , "img_name"]
labelsTest = labelsTest.astype({'face_shape': 'int32'})
labelsTest = labelsTest.astype({'img_name': 'string'})

In [3]:
#Load training set################################
imagesTrain = np.zeros((10000, 150 , 150, 3)) 

for i in range(0,10000):
    image = cv.imread('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set/img/' + labelsTrain["img_name"][i])
    image = image[75:420,95:405] # cut 95 from left / right sides , 75 from top, 80 from the bottom
    image = cv.resize(image, dsize=(150 , 150), interpolation=cv.INTER_CUBIC)
    # imagesTrain[i] = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    imagesTrain[i] = image
#imagesTrain = imagesTrain.reshape(10000,26815)    
#imagesTrain = pd.DataFrame(imagesTrain) 


#Load Testing set################################
imagesTest = np.zeros((2500, 150 , 150, 3))

for i in range(0,2500):
    image = cv.imread('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set_test/img/' + labelsTest["img_name"][i])
    image = image[75:420,95:405]
    image = cv.resize(image, dsize=(150 , 150), interpolation=cv.INTER_CUBIC)
    #imagesTest[i] = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    imagesTest[i] = image
    
#imagesTest = imagesTest.reshape(2500,26815)   
#imagesTest = pd.DataFrame(imagesTest) 

In [27]:
tf.config.run_functions_eagerly(True)
#needed for tf to run

In [4]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten
print("num of gpus available: ",len(tf.config.experimental.list_physical_devices('GPU')))
from keras.utils import normalize,to_categorical
from keras.preprocessing.image import ImageDataGenerator

num of gpus available:  1


In [6]:
labelsTrain = labelsTrain['face_shape']
labelsTest = labelsTest['face_shape']

imagesTrain = normalize(imagesTrain,axis=1)
imagesTest = normalize(imagesTest,axis=1)

labelsTrain = to_categorical(labelsTrain)
labelsTest = to_categorical(labelsTest)

In [7]:
datagen_train = ImageDataGenerator()
#needed to save memory

datagen_Test = ImageDataGenerator()


In [29]:
model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add( Dense(128,activation='relu') )
model.add( Dense(64,activation='relu') )
model.add( Dense(5,activation='softmax') )


In [30]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 36, 36, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 17, 17, 128)     

In [31]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [32]:
history = model.fit( 
    datagen_train.flow(imagesTrain, labelsTrain, batch_size=100),
    epochs=10,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=datagen_Test.flow(imagesTest, labelsTest, batch_size=100),
)

Epoch 1/10
100/100 [==============================] - 13s 117ms/step - loss: 0.9690 - accuracy: 0.5808 - val_loss: 0.0736 - val_accuracy: 0.9856
Epoch 2/10
100/100 [==============================] - 12s 117ms/step - loss: 0.0421 - accuracy: 0.9888 - val_loss: 0.0086 - val_accuracy: 0.9980
Epoch 3/10
100/100 [==============================] - 12s 116ms/step - loss: 0.0101 - accuracy: 0.9971 - val_loss: 0.0354 - val_accuracy: 0.9900
Epoch 4/10
100/100 [==============================] - 12s 116ms/step - loss: 0.0074 - accuracy: 0.9978 - val_loss: 0.0033 - val_accuracy: 0.9988
Epoch 5/10
100/100 [==============================] - 12s 116ms/step - loss: 0.0012 - accuracy: 0.9996 - val_loss: 0.0019 - val_accuracy: 0.9996
Epoch 6/10
100/100 [==============================] - 12s 117ms/step - loss: 0.0023 - accuracy: 0.9992 - val_loss: 0.0029 - val_accuracy: 0.9992
Epoch 7/10
100/100 [==============================] - 12s 118ms/step - loss: 0.0012 - accuracy: 0.9997 - val_loss: 4.9524e-04 - va